In [1]:
from BRScraper import nba
import pandas as pd 
import numpy as np

In [2]:
def select_data(desired_seasons, stats, salary, cap):
    df_f = pd.DataFrame()

    for desired_season in desired_seasons:
        # Selecting only salaries of S and S - 1
        salary1 = salary[['Player', desired_season,
                          str(int(desired_season[:4])-1)+'-'+str(int(desired_season[5:7])-1)]]
        
        stats1 = stats[stats["Season S-1"]==str(int(desired_season[:4])-1)+'-'+str(int(desired_season[5:7])-1)]

        df = stats1.merge(salary1, on='Player', how='left', validate='1:1')

        df = df.rename(columns={str(int(desired_season[:4])-1)+'-'+str(int(desired_season[5:7])-1):'Salary S-1',
                                  desired_season:'Salary S'})
        
        # Removing null values
        df = df[(df['Salary S']!=-1) & (df['Salary S'].notna())]

        df['Year S'] = ('20'+((df['Season S-1'].str[5:7]).astype(int)+1).astype(str)).astype(int)
        df['Year S-1'] = ('20'+(df['Season S-1'].str[5:7]).astype(str)).astype(int)

        cap2 = cap.rename(columns={'Cap Maximum':'Cap Maximum S', 'Year':'Year S'})
        df = df.merge(cap2, how='left', on='Year S', validate='m:1')

        cap2 = cap2.rename(columns={'Cap Maximum S':'Cap Maximum S-1', 'Year S':'Year S-1'})
        df = df.merge(cap2, how='left', on='Year S-1', validate='m:1')

        df['% of Cap S-1'] = df['Salary S-1']/df['Cap Maximum S-1']*100
        df['% of Cap S-1'][df['% of Cap S-1']<0] = -1

        df['% of Cap S'] = df['Salary S']/df['Cap Maximum S']*100
        df['% of Cap S'][df['% of Cap S']<0] = -1

        # Get dummies for positions
        pos = pd.get_dummies(df['Pos']).astype(int)

        for col in pos.columns:
            if len(col)>3:
                pos = pos.rename(columns={col: col.split('-')[0]})
        pos = pos[['PG','SG','SF','PF','C']]
        pos = pos.groupby(pos.columns, axis=1).sum()

        df = df.join(pos)
        df = df.drop(columns={'Pos','Year S','Year S-1'}).reset_index(drop=True)

        df_f = pd.concat([df_f, df], ignore_index=True)
        
    df_f = df_f.fillna(0)
        
    return df_f
        